In [2]:
import pdb
import numpy as np
from tables import *
import gcp.bigquery as bq
import gcp.storage as storage
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
try:
   import cPickle as pickle
except:
   import pickle
EST_PICKLE_FILENAME = 'baseline_final_estimator.pkl'
FIELDS_PICKLE = 'fields-4.pkl'

seed = 13
np.random.seed(seed)

fields = pickle.load(open(FIELDS_PICKLE, "r") )
features = fields[1:]

# Use this instead of len(features) since this variable can change
# e.g. when one hot encoding is used and/or new features are added.
n_features = len(features)
datafile_path = 'xjk_pytable.h5'

print "Number of features: {}".format(len(features))
print "Features:"
print features

IOError: [Errno 2] No such file or directory: 'fields-4.pkl'

# Load Dataset

In [10]:
fileh1 = open_file(datafile_path, mode = 'r')

object = fileh1.get_node('/train', 'gaps')
object_array_data = object.read()
fileh1.close()

# Convert to vectorized array that we can use in further processing.
all_data = np.zeros((object_array_data.shape[0], len(fields)))
print 'there are {} rows'.format(object_array_data.shape[0])
for rcounter, row in enumerate(object_array_data):
  for fcounter, field in enumerate(fields):
    all_data[rcounter, fcounter] = row[field]
  if rcounter % 5000 == 0:
    print 'processed {} rows'.format(rcounter)
all_data_original = np.copy(all_data)

data = all_data[:,1:]
targets = all_data[:,0]

there are 102592 rows
processed 0 rows
processed 5000 rows
processed 10000 rows
processed 15000 rows
processed 20000 rows
processed 25000 rows
processed 30000 rows
processed 35000 rows
processed 40000 rows
processed 45000 rows
processed 50000 rows
processed 55000 rows
processed 60000 rows
processed 65000 rows
processed 70000 rows
processed 75000 rows
processed 80000 rows
processed 85000 rows
processed 90000 rows
processed 95000 rows
processed 100000 rows


# One Hot Encoding

One hot encoding is s

In [38]:
from sklearn.preprocessing import OneHotEncoder
one_hot = OneHotEncoder(categorical_features=[0, 1, 2, 3, 4], sparse=False)
one_hot.fit(data_train)

OneHotEncoder(categorical_features=[0, 1, 2, 3], dtype=<type 'float'>,
       handle_unknown='error', n_values='auto', sparse=False)

In [39]:
one_hot.n_values_

array([4, 5, 5, 5])

In [40]:
one_hot.feature_indices_

array([ 0,  4,  9, 14, 19])

In [41]:
data_train = one_hot.transform(data_train_original)
data_test = one_hot.transform(data_test_original)
n_features = data_train.shape[1]
print 'new number of features: {}'.format(n_features)

new number of features: 169


# Scorer Creation (MAPE)

In [2]:
def mape(y, predictions):
#   num_timeslots = 43
#   num_districts = 66
  if len(y.shape) == 1:
    y = np.asmatrix(y)
  if len(predictions.shape) == 1:
    predictions = np.asmatrix(predictions)
  y = y.astype(float)
  predictions = predictions.astype(float)
  return np.mean(np.absolute((y-predictions)/y))

# from keras import backend as K

# def mape(y, predictions):
#   return K.mean(K.abs(y-predictions/K.clip(K.abs(y), K.epsilon(), np.inf)), axis=-1)

mape_scorer = make_scorer(mape, greater_is_better=False)

Testing MAPE

In [3]:
from sklearn.linear_model import LogisticRegression

predictions = np.array([1.0, 2.0, 3.0, 4.0]).astype('float32')
y = np.array([1.0, 2.0, 3.0, 4.0]).astype('float32')

# Should return 0.0
print mape(y, predictions)

# Should return higher score
predictions = np.array([1.0, 2.0, 2.0, 3.0]).astype('float32')
print(mape(y, predictions))

# Should return highest score
predictions = np.array([1000.0, 22.0, 11.0, 31.0]).astype('float32')
print(mape(y, predictions))

# est = LogisticRegression()
# X = np.random.rand(10,4)
# y = X.sum(axis=1)
# est.fit(X, y)
# predictions = est.predict(X)
# print(mape(y, predictions))

0.0
0.145833333333
254.604166667


# Building and Testing Algorithm(s)

In [42]:
from sklearn.base import BaseEstimator, RegressorMixin
from scipy.sparse import coo_matrix, hstack

class CustomRegressor(BaseEstimator, RegressorMixin):
  def __init__(self):
    pass

  def fit(self, X, y):
#     self.classes_, indices = np.unique(["foo", "bar", "foo"],
#                                     return_inverse=True)
#     self.majority_ = np.argmax(np.bincount(indices))
    return self

  def predict(self, X):
    # 56: gap_1_slots_ago
    # 58: gap_2_slots_ago
    # 60: gap_3_slots_ago
#     X = X.tocsr()
#     v1 = coo_matrix(np.asmatrix(np.ones(X.shape[0])).T)
    v1 = np.asmatrix(np.ones(X.shape[0]))
    v2 = np.asmatrix((X[:, 23]*0.65+X[:, 25]*0.25+X[:, 27]*0.15)/2)
    predictions = np.asarray(np.concatenate((v1, v2), axis=0).max(axis=0))
    
    return predictions
  
custom_est = CustomRegressor()
custom_est.fit(data_train_original, data_test_original)
custom_predictions = custom_est.predict(data_test_original)
print(mape(targets_test, custom_predictions))

0.39539351749


In [43]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.preprocessing import Imputer, OneHotEncoder
from sklearn.grid_search import RandomizedSearchCV
from sklearn.grid_search import GridSearchCV
from sklearn.decomposition import PCA

steps = [
  ('one_hot', OneHotEncoder(categorical_features=[0, 1, 2, 3], sparse=False)),
#   ('impute', Imputer(0)),
#   ('feature_selection', SelectKBest(f_classif)),
  ('pca', PCA(n_components=120)),
  ('estimate', DecisionTreeRegressor())
]

est = Pipeline(steps)

params = {
#   'one_hot__n_values': [7, 10, 20],
#   "feature_selection__k": [i for i in range(1, n_features - 1)]
  'estimate__max_features': [i for i in range(110, 120)],
#   'estimate__learning_rate': [0.1, 0.5, 1, 5, 10],
#   'estimate__n_estimators': [i for i in range(110, 120, 2)],
#   'estimate__loss': ['linear', 'square', 'exponential']
}
# cross_validation_iter = StratifiedShuffleSplit(y=targets_train, test_size=0.3,
#                                                random_state=RANDOM_STATE, n_iter=10)
# search_params = RandomizedSearchCV(
#   estimator=est,
#   param_distributions=params,
#   cv=5,
#   scoring=mape_scorer,
#   n_jobs=2,
#   verbose=1
# )

search_params = GridSearchCV(
  estimator=est,
  param_grid=params,
  cv=5,
  scoring=mape_scorer,
  n_jobs=5,
  verbose=3
)

search_params.fit(data_train_original, targets_train)
print(search_params.grid_scores_)
print(search_params.best_params_)
print(search_params.best_score_)
search_params.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] estimate__max_features=110 ......................................
[CV] estimate__max_features=110 ......................................
[CV] estimate__max_features=110 ......................................
[CV] estimate__max_features=110 ......................................
[CV] estimate__max_features=110 ......................................
[CV] ............ estimate__max_features=110, score=-1.015761 -   0.9s
[CV] estimate__max_features=111 ......................................
[CV] ............ estimate__max_features=110, score=-0.997429 -   0.9s
[CV] estimate__max_features=111 ......................................
[CV] ............ estimate__max_features=110, score=-1.007373 -   0.9s
[CV] estimate__max_features=111 ......................................
[CV] ............ estimate__max_features=110, score=-0.774039 -   1.0s
[CV] estimate__max_features=111 ......................................
[CV] ...........

[Parallel(n_jobs=5)]: Done   1 jobs       | elapsed:    0.9s


[CV] estimate__max_features=111 ......................................
[CV] ............ estimate__max_features=111, score=-1.153101 -   0.8s
[CV] estimate__max_features=112 ......................................
[CV] ............ estimate__max_features=111, score=-1.121552 -   0.8s
[CV] estimate__max_features=112 ......................................
[CV] ............ estimate__max_features=111, score=-0.903246 -   0.9s
[CV] estimate__max_features=112 ......................................
[CV] ............ estimate__max_features=111, score=-0.817320 -   0.9s
[CV] estimate__max_features=112 ......................................
[CV] ............ estimate__max_features=111, score=-1.487765 -   0.9s
[CV] estimate__max_features=112 ......................................
[CV] ............ estimate__max_features=112, score=-1.011377 -   0.9s
[CV] ............ estimate__max_features=112, score=-0.706411 -   0.8s
[CV] estimate__max_features=113 ......................................
[CV] e

[Parallel(n_jobs=5)]: Done  32 jobs       | elapsed:    6.2s


[CV] ............ estimate__max_features=116, score=-1.271926 -   0.8s
[CV] estimate__max_features=117 ......................................
[CV] ............ estimate__max_features=117, score=-1.026548 -   0.9s
[CV] estimate__max_features=118 ......................................
[CV] ............ estimate__max_features=117, score=-0.746136 -   0.8s
[CV] estimate__max_features=118 ......................................
[CV] ............ estimate__max_features=117, score=-1.111784 -   0.8s
[CV] ............ estimate__max_features=117, score=-1.270591 -   0.8s
[CV] estimate__max_features=118 ......................................
[CV] estimate__max_features=118 ......................................
[CV] ............ estimate__max_features=117, score=-1.301783 -   0.8s
[CV] estimate__max_features=118 ......................................
[CV] ............ estimate__max_features=118, score=-1.043956 -   0.7s
[CV] estimate__max_features=119 ......................................
[CV] .

[Parallel(n_jobs=5)]: Done  42 out of  50 | elapsed:    8.1s remaining:    1.5s


[CV] ............ estimate__max_features=119, score=-1.013921 -   0.8s
[CV] ............ estimate__max_features=119, score=-0.742392 -   0.7s
[CV] ............ estimate__max_features=119, score=-1.124674 -   0.6s
[CV] ............ estimate__max_features=119, score=-1.101916 -   0.8s
[CV] ............ estimate__max_features=119, score=-1.400818 -   0.6s


[Parallel(n_jobs=5)]: Done  50 out of  50 | elapsed:    8.9s finished


[mean: -1.02255, std: 0.17324, params: {'estimate__max_features': 110}, mean: -1.09660, std: 0.23333, params: {'estimate__max_features': 111}, mean: -0.98646, std: 0.15590, params: {'estimate__max_features': 112}, mean: -1.11270, std: 0.17288, params: {'estimate__max_features': 113}, mean: -1.02774, std: 0.17128, params: {'estimate__max_features': 114}, mean: -1.09007, std: 0.20414, params: {'estimate__max_features': 115}, mean: -1.02891, std: 0.16819, params: {'estimate__max_features': 116}, mean: -1.09137, std: 0.20010, params: {'estimate__max_features': 117}, mean: -1.05021, std: 0.28213, params: {'estimate__max_features': 118}, mean: -1.07674, std: 0.21152, params: {'estimate__max_features': 119}]
{'estimate__max_features': 112}
-0.986455231943


Pipeline(steps=[('one_hot', OneHotEncoder(categorical_features=[0, 1, 2, 3], dtype=<type 'float'>,
       handle_unknown='error', n_values='auto', sparse=False)), ('pca', PCA(copy=True, n_components=120, whiten=False)), ('estimate', DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=112,
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, random_state=None,
           splitter='best'))])

Test data's prediction MAPE score:

In [44]:
final_est = search_params.best_estimator_
test_predictions = final_est.predict(data_test_original)
print(mape(targets_test, test_predictions))

0.923959635952


In [45]:
pickle.dump(final_est, open(EST_PICKLE_FILENAME, "w") )

Run "Process Final Test Data With Final Algorithm" to use pickled final algorithm against final test data to produce csv required by this competition.

In [46]:
from sklearn.tree import DecisionTreeRegressor
# Just testing Imputer. Turns out somehow Imputer causes number of features reduced, weird.

# imputer = Imputer()
est = DecisionTreeRegressor(max_features=len(features))

data_train_i = np.copy(data_train)
print(data_train.shape)
print(data_train[0:10])
# data_train_i = imputer.fit_transform(data_train)
data_train_i[np.isnan(data_train_i)] = 0
data_train_i.astype('float32')
print(data_train_i.shape)
print(data_train_i[0:10])
est.fit(data_train_i, targets_train)
predictions = est.predict(data_test)
print(mape(data_test, predictions, targets_test))

(800, 169)
[[  1.00000000e+00   0.00000000e+00   1.00000000e+00 ...,   1.74300000e+03
    2.07500000e+03   0.00000000e+00]
 [  1.00000000e+00   1.00000000e+00   0.00000000e+00 ...,   1.66000000e+02
    7.47000000e+02   1.66000000e+02]
 [  1.00000000e+00   0.00000000e+00   1.00000000e+00 ...,   1.90900000e+03
    3.40300000e+03   0.00000000e+00]
 ..., 
 [  1.00000000e+00   0.00000000e+00   1.00000000e+00 ...,   7.30400000e+03
    9.04700000e+03   0.00000000e+00]
 [  1.00000000e+00   0.00000000e+00   1.00000000e+00 ...,   5.89300000e+03
    9.62800000e+03   0.00000000e+00]
 [  1.00000000e+00   0.00000000e+00   1.00000000e+00 ...,   7.30400000e+03
    9.04700000e+03   0.00000000e+00]]
(800, 169)
[[  1.00000000e+00   0.00000000e+00   1.00000000e+00 ...,   1.74300000e+03
    2.07500000e+03   0.00000000e+00]
 [  1.00000000e+00   1.00000000e+00   0.00000000e+00 ...,   1.66000000e+02
    7.47000000e+02   1.66000000e+02]
 [  1.00000000e+00   0.00000000e+00   1.00000000e+00 ...,   1.90900000e+03

TypeError: mape() takes exactly 2 arguments (3 given)

# Results

Pipeline(steps=[('one_hot', OneHotEncoder(categorical_features=[0, 1, 2, 3], dtype=<type 'float'>,
       handle_unknown='error', n_values='auto', sparse=False)), ('pca', PCA(copy=True, n_components=120, whiten=False)), ('estimate', DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=112,
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, random_state=None,
           splitter='best'))])
           
1000 training data, Score: 0.924

Pipeline(steps=[('one_hot', OneHotEncoder(categorical_features=[0, 1, 2, 3], dtype=<type 'float'>,
       handle_unknown='error', n_values='auto', sparse=False)), ('pca', PCA(copy=True, n_components=120, whiten=False)), ('estimate', DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=114,
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, random_state=None,
           splitter='best'))])

5000 training data, Score: 0.992


Pipeline(steps=[('one_hot', OneHotEncoder(categorical_features=[0, 1, 2, 3], dtype=<type 'float'>,
       handle_unknown='error', n_values='auto', sparse=False)), ('pca', PCA(copy=True, n_components=120, whiten=False)), ('estimate', DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=111,
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, random_state=None,
           splitter='best'))])
           
20000 training data, Score: 1.001